In [ ]:
#316
#Given an array of length N,
#where each element i represents the number of ways we can produce i units of change.
#For example, [1, 0, 1, 1, 2] would indicate that there is only one way to make
#0, 2 or 3 units, and two ways of making 4 units.

#Given such an array, determine the denominations that must be in use.
#In the case above, for example, there must be coins with value 2,3 and 4.

In [ ]:
#1. THE BRUTE FORCE method.
def get_powerset(nums):
    result = [[]]
    
    for x in nums:
        result.extend([subset + [x] for subset in result])
        
    return result

def change_combinations(coins, n):
    ways = [1] + [0] * n
    
    for coin in coins:
        for i in range(coin, n + 1):
            ways[i] += ways[i - coin]
            
        return ways
    
def find_denomination(array):
    nonzero = [i for i, val in enumerate(array[1:], 1) if val > 0]
    powerset = get_powerset(nonzero)
    
    for coins in powerset:
        ways = change_combinations(coins, len(array) - 1)
        if ways == array:
            return coins
        
#as a result,
#the time complexity of this approach will be O(N * 2^^N).
#This solution will require O(2^^N) to store all the subsets.

#Our final note is that we must take care not to double count when iterating through
#our previous coins.
#For example, if i = 7, and 3 and 4 are denominations in out solution set,
#This only represents one way of producing 7.
#To handle this case, we will add logic so that when two coints in our solution sum to an index,
#only the lower one will cause us to decrement the value at that index.

def find_deniminations(array):
    coins = set()
    
    for i, val in enumerate(array[1:], 1):
        if val > 0:
            for coin in coins:
                if array[(i - coin)] > 0 and ((i - coin) not in coins or (i - coin) <= coin):
                        val -= 1
                if val > 0:
                    coins.add(i)
                    
    return coins

#THe time complexity of this algorithm is O(M *N),
#where M is the number of coins in our solution and N is the length of our input
#The space complexity will be O(M), since we require extra space to store the set of solution coins.


In [ ]:
#318

#You are going on a road trip,
#and would like to create a suitable music playlist.
#The trip will require N songs, though you only have M songs downloaded,
#where M < N. 
#A valid playlist should select each song at least once,
#and guarantee a buffer of B songs between repeats
#Given N,M, and B, determine the number of valid playlists.

In [6]:
#Step by step
#The easier step should be done earlier than others.
#1. Creating playlists of N songs from M downladed options, but without any buffer requirement

#Dynamic programming
#ways[i][j] represents the number of ways of making a playlist of length i with j unique songs.

#If the last song is new
#ways[i - 1][j - 1]

#if the last song is a repeat
#ways[i - 1][j]

#once we built up our matrix in this way, we can take tha value of ways[n][m] to be our solution.

def valid_playlists(n, m, b):
    ways = [[0 for _ in range(m + 1)] for _ in range(n + 1)]
    ways[0][0] = 1
    
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            ways[i][j] = ways[i - 1][j - 1] * (m - (j - 1)) + ways[i - 1][j] * j
            
    return ways[n][m]

valid_playlists(50, 41, 10)

70638297642189060798456634632900899224219226941200244817264640000000000

In [16]:
#Now let us try adding in the buffer B to our dynamic programming formula.
#If the last song is new, no change needs to be made, since it cannot possibly be a repeat.

#If the song is old, on the other hand, there will be B options that cannot be used as the
#next song, specifically the last B songs in our playlist. Therefore the number of new playlist
#formed can be represented as ways[i - 1][j] & (j - b). 
#If the buffer is bigger than the number of distinct songs played so far, no repeat songs
#are possible, so we can use max(j - b, 0) to handle this case.

def valid_playlists2(n, m, b):
    ways = [[0 for _ in range(m +1)] for _ in range(n + 1)]
    ways[0][0] = 1
    
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            ways[i][j] = ways[i - 1][j - 1] * (m - (j -1)) + ways[i - 1][j] * max(j - b, 0)
            
    return ways[n][m]

#The time and space complexity of this algorithm will be 
#O(M * N), since we must loop through each cell of our M X N matrix and perform a few calculations.

valid_playlists2(20, 14, 1)

252157903368763392000

## 323 requires strong mathematical background

In [ ]:
#323
#create an algorithm to efficiently compute the approximate median of a list of numbers

#More precisely, given an unordered list of N numbers, find an element whose rank is
#between N / 4 and 3 * N / 4, with a high level of certainty, in less than O(N) time.

In [ ]:
#We will need some (hand-wavy) mathematical background.

from math import log
from random import sample

def find_median(array, precision):
    m = precision * int(log(len(array, 2)))
    
    subset = sample(array, m)
    subset.sort()
    
    return subset[m // 2]

In [2]:
#324
#amazon

#there are N mice and N holes placed at integer points along a line.
#Given this, find a method that maps mice to holes such that the largest number of steps
#any mouse takes is minimized.

#Each move consists of moving one mouse one unit to the left or right, and only one mouse can fit inside each hole.

#[1, 4, 9, 15] mice
#[10, -5, 0, 16] holes
#in this case, the best pairing would require us to send the mouse at 1 to the hole at -5,
#so our function should return 6.

In [1]:
#Brute force
from itertools import permutations

def send(mice, holes):
    moves = float('inf')
    
    orders = permutations(holes, len(mice))
    for order in orders:
        max_steps = 0
        for mouse, hole in zip(mice, order):
            max_steps = max(max_steps, abs(mouse - hole))
        moves = min(moves, max_steps)
        
    return moves

#This is grossly inefficient, since generating all the permutations is exponential in the number of holes


In [8]:
#finally, note that every alternative mapping between mice and holes can be derived from our
#greedy solution using a series of swaps. Since each one of these swaps cannot imporve our result,
#It is impossible to combine them to arrive at a more optimal result.

def send2(mice, holes):
    mice.sort()
    holes.sort()
    
    moves = 0
    for i in range(len(mice)):
        moves = max(moves, abs(mice[i] - holes[i]))
        
    return moves

mice = [4, 6, 9, 15]
holes = [10 , -3, 0, 16]

send2(mice, holes)
#The time complexity of this solution is O(N log N), since we must first sort our lists
#of mice and holes. The space required is O(1), since we need only iterate over our
#input and update the max counter.

7